# Регрессия - пора творить!

Если вы дошли до этой практики и выполнили все предыдущие - вау! Аплодисменты вам!

<p align="center"><img src="https://raw.githubusercontent.com/AleksDevEdu/ml_edu/master/assets/leo.png" width=500/></p>



Тем не менее, не время расслабляться! На этот раз мы не будем рассказывать вам новый материал - лишь дадим пару напутствий и пожелаем успехов и сил!

В этот раз вам предлагается разработать модель предсказания цены домов по набору данных Boston Houses Pricing. Мы его уже немного пощупали в прошлой практике, но в этой вам предстоит сделать всё самостоятельно и на полных данных!

Что надо будет сделать:
- загрузить данные;
- посмотреть и размер, и примеры данных;
- проверить количество пропусков в данных;
    - убедиться, что пропусков в данных нет и можно строить **baseline**;
- разделить данные на train и test;
    - Делаем это один раз! Все дальнейшие модификации и улучшения должны делаться на основе единственного разделения!
    - И не забудьте зафиксировать сид! Переменную `RANDOM_SEED` мы даём ниже;
- создать baseline модель, получить метрики на тестовой выборке;
- проанализировать данные, подготовить план по очистке и предобработке данных;
- создать функцию очистки;
- произвести очистку train и test выборок -> обучаем модель с очищенными данными;
    - Снимаем метрики и так мы оцениваем влияние очистки данных на обучение;
- создать класс предобработки данных;
- сделать предобработку очищенных данных -> обучаем модель;
    - Снимаем метрики и так мы оцениваем влияние предобработки данных на обучение;

А дальше можете пофантазировать, почитать интернет и посмотреть, как улучшить метрики!

**Ваша цель - достигнуть наилучших показателей!**

Вот вам пара идей по интересной организации предобработке:
- по признаку `CRIM` создать признак `is_CRIM`, который будет равен 1, если `CRIM` не равен 0 и 0, если `CRIM` равен нулю;
- сделать обучение модели [Lasso](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html), посмотреть, какие признаки имеют веса близкие к 0 и убрать их из обучения как незначимые;
- оставить только признаки с высокой корреляцией с целевой переменной;
- посмотреть, какие переменные коррелирцют между собой и оставить только единственные из пар коррелирующих;
- посмотреть, в каких примерах данных модель ошибается сильнее всего (отсортировать записи из теста по величине ошибки) и проанализировать, есть ли какая-то зависимость или шум;
- комбинировать разные подходы в зависимости от того, помогает это предсказаниям или нет.

Ещё большим плюсом будет, если вы покажете, как влияет каждый этап предобработки на результаты обучения!

In [1]:
# Зафиксируем состояние случайных чисел
import numpy as np
import random

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)

Что мы ждем от вас?

Готовый ноутбук с таким же названием, в котором будут обязательные пункты с выводами по анализу данных и анализу поведения модели, а также дополнительные пункты с вашими мыслями и результатами улучшения модели.

Успехов вам, мы в вас верим, вы точно справитесь!

<p align="center"><img src="https://raw.githubusercontent.com/AleksDevEdu/ml_edu/master/assets/cat-computer-peek-out-36960208.jpg" width=600/></p>

## Загрузка датасета

In [2]:
import pandas as pd

data = pd.read_csv('boston.csv')

### Посмотрим на пример данных

In [4]:
data.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222.0,18.7,396.90,5.33,36.2


### Посмотрим на информацию о данных и проверим, есть ли пропуски в данных

In [5]:
print(data.info())
print('\n')
print(data.isnull().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 14 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   CRIM     506 non-null    float64
 1   ZN       506 non-null    float64
 2   INDUS    506 non-null    float64
 3   CHAS     506 non-null    int64  
 4   NOX      506 non-null    float64
 5   RM       506 non-null    float64
 6   AGE      506 non-null    float64
 7   DIS      506 non-null    float64
 8   RAD      506 non-null    int64  
 9   TAX      506 non-null    float64
 10  PTRATIO  506 non-null    float64
 11  B        506 non-null    float64
 12  LSTAT    506 non-null    float64
 13  MEDV     506 non-null    float64
dtypes: float64(12), int64(2)
memory usage: 55.5 KB
None


CRIM       0
ZN         0
INDUS      0
CHAS       0
NOX        0
RM         0
AGE        0
DIS        0
RAD        0
TAX        0
PTRATIO    0
B          0
LSTAT      0
MEDV       0
dtype: int64


### Супер, пропусков нет! Все наглядные графики уже были построены на прошлой практике. Теперь будем строить baseline

In [6]:
from sklearn.model_selection import train_test_split

# Разделим данные на тренировочную и тестовую выборку
RANDOM_SEED = 42
X, y = data.drop(columns='MEDV'), data['MEDV']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)


### Для начала попробуем обучить простую линейную регрессию

In [7]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

model = LinearRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
print(f"Baseline MSE: {mse}")


Baseline MSE: 24.291119474973538


Можно использовать корень из mse, чтобы было нагляднее:

In [8]:
print(f"Rooted MSE: {mse ** (0.5)}")

Rooted MSE: 4.928602182665339


В целом сойдет, но хочется лучше!

### Проанализируем данные

1. **Создание признаков:** по признаку CRIM можно создать бинарный признак is_CRIM, который будет 1, если CRIM не равен 0.

2. **Предобработка данных:** вероятнее всего, будет полезно нормализовать числовые данные

3. **Удаление выбросов:** напишем функцию для чистки данных? Почему бы и нет

Функцию для очистки возьмем из прошлой практики

In [9]:
def clean_dataset(df):
    outliers_mask_1 = df["MEDV"] == 50
    outliers_mask_2 = df["RM"] < 4
    outliers_mask_3 = (df["RM"] > 8) & (df["MEDV"] < 30)

    outliers_mask = outliers_mask_1 | outliers_mask_2 | outliers_mask_3

    df_cleaned = df.loc[~outliers_mask]

    return df_cleaned


In [10]:
# почистим наши данные!

data_cleaned = clean_dataset(data)

In [11]:
# далее будем работать с чистыми
X, y = data_cleaned.drop(columns='MEDV'), data_cleaned['MEDV']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)

Посмотрим, как изменилась mse?

In [12]:
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
mse_cleaned = mean_squared_error(y_test, y_pred)

print(f"MSE после очистки: {mse_cleaned}")
print(f"Rooted MSE после чистки: {mse_cleaned ** (0.5)}")

MSE после очистки: 12.583244664773558
Rooted MSE после чистки: 3.547286944239718


Показатели действительно улучшились!

## Предобработка данных

Возьмем класс из практики! Не зря же мы над ним так долго трудились

In [13]:
from sklearn.preprocessing import MinMaxScaler


class DataPreprocessing:
    def __init__(self):
        self.min_max_scaler = MinMaxScaler()

    def fit(self, df) -> None:
        df_copy = df.copy()

        df_copy['DIS_log'] = np.log(df_copy['DIS'])
        df_copy = df_copy.drop('DIS', axis=1)

        df_copy['LSTAT_poly_2'] = df_copy['LSTAT'] ** 2

        self.min_max_scaler.fit(df_copy)

    def transform(self, df) -> pd.DataFrame:
        df_copy = df.copy()

        df_copy['DIS_log'] = np.log(df_copy['DIS'])
        df_copy = df_copy.drop('DIS', axis=1)

        df_copy['LSTAT_poly_2'] = df_copy['LSTAT'] ** 2
        df_copy[df_copy.columns] = self.min_max_scaler.transform(df_copy)

        return df_copy


Обработаем данные =)

In [14]:
preprocessor = DataPreprocessing()

preprocessor.fit(X_train)

X_train_transformed = preprocessor.transform(X_train)
X_test_transformed = preprocessor.transform(X_test)



Посмотрим, как улучшились метрики?

In [15]:
model = LinearRegression()
model.fit(X_train_transformed, y_train)

y_pred = model.predict(X_test_transformed)

mse = mean_squared_error(y_test, y_pred)

print(f'Mean Squared Error: {mse}')
print(f"Rooted MSE после чистки: {mse_cleaned ** (0.5)}")

Mean Squared Error: 11.052930609934448
Rooted MSE после чистки: 3.547286944239718


В целом, стало еще лучше!

# **Fantasy and Improvement**

Заменим признак CRIM на бинарный is_CRIM

In [16]:
X_train_transformed['CRIM'] = (X_train_transformed['CRIM'] != 0).astype(int)
X_test_transformed['CRIM'] = (X_test_transformed['CRIM'] != 0).astype(int)

Давайте теперь обучим Lasso регрессию

In [17]:
from sklearn.linear_model import Lasso

lasso = Lasso(0.001)
lasso.fit(X_train_transformed, y_train)

Lasso(alpha=0.001)

Посмотрим на коэффициенты, что с ними?

In [18]:
coefficients = pd.Series(lasso.coef_, index=X_train_transformed.columns)

print(coefficients)

CRIM            -2.165404
ZN              -0.048021
INDUS            0.517520
CHAS             1.469153
NOX             -5.667948
RM              17.679142
AGE             -0.587296
RAD              3.202360
TAX             -5.980824
PTRATIO         -6.562859
B                3.768127
LSTAT          -37.092327
DIS_log         -7.748834
LSTAT_poly_2    26.235941
dtype: float64


Многие занулились. Это означает, что они принимают мало участия в формировании целевой переменной. Уберем их?

In [19]:
important_features = coefficients[abs(coefficients) > 1e-4].index

print(important_features)

Index(['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'RAD', 'TAX',
       'PTRATIO', 'B', 'LSTAT', 'DIS_log', 'LSTAT_poly_2'],
      dtype='object')


In [20]:
X_train_filtered = X_train_transformed[important_features]
X_test_filtered = X_test_transformed[important_features]

In [21]:
X_train_filtered.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,RAD,TAX,PTRATIO,B,LSTAT,DIS_log,LSTAT_poly_2
434,1,0.0,0.636089,0.0,0.674897,0.492975,0.948507,1.000000,0.914122,0.808511,0.252938,0.366491,0.283103,0.157330
325,1,0.0,0.231408,0.0,0.222222,0.546082,0.121524,0.173913,0.190840,0.744681,0.991881,0.086135,0.659462,0.015222
25,1,0.0,0.260098,0.0,0.314815,0.347940,0.852729,0.130435,0.229008,0.893617,0.764285,0.403723,0.576909,0.186855
230,1,0.0,0.186863,0.0,0.244856,0.438914,0.671473,0.304348,0.229008,0.510638,0.953225,0.268686,0.495229,0.091669
309,1,0.0,0.326538,0.0,0.327160,0.436771,0.760041,0.130435,0.223282,0.617021,0.998336,0.222006,0.424087,0.066407


Теперь мы можем обучить на получившемся датасете Ridge регрессию. Т.е. регрессию с L2 регуляризацией

In [22]:
from sklearn.linear_model import Ridge

ridge = Ridge(alpha=0.001)
ridge.fit(X_train_filtered, y_train)

y_pred = ridge.predict(X_test_filtered)

mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error after Ridge: {mse}')
print(f'Rooted MSE: {mse ** 0.5}')

Mean Squared Error after Ridge: 10.544437681108791
Rooted MSE: 3.247219992718201


### **Получилось супер! Наши махинации с признаками помогли улучшить результат.**